In [9]:
!pip install transformers datasets evaluate

In [10]:
!pip install --upgrade datasets transformers huggingface_hub

In [11]:
from huggingface_hub import notebook_login
notebook_login()

In [12]:
# from datasets import load_dataset
# dataset = load_dataset("imdb")
from datasets import load_dataset
dataset = load_dataset("imdb", split={"train": "train", "test": "test"})

In [3]:
from transformers import DistilBertTokenizerFast

tokenizer = DistilBertTokenizerFast.from_pretrained("distilbert-base-uncased")

def tokenize_function(example):
    return tokenizer(example["text"], truncation=True, padding="max_length", max_length=512)

tokenized_datasets = dataset.map(tokenize_function, batched=True)
tokenized_datasets = tokenized_datasets.remove_columns(["text"])
tokenized_datasets.set_format("torch")


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/483 [00:00<?, ?B/s]

Map:   0%|          | 0/25000 [00:00<?, ? examples/s]

Map:   0%|          | 0/25000 [00:00<?, ? examples/s]

In [4]:
from transformers import DistilBertForSequenceClassification
model = DistilBertForSequenceClassification.from_pretrained("distilbert-base-uncased", num_labels=2)

model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [18]:
from transformers import TrainingArguments, Trainer
import evaluate
import numpy as np

accuracy = evaluate.load("accuracy")

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=1)
    return accuracy.compute(predictions=predictions, references=labels)

training_args = TrainingArguments(
    output_dir="./results",
    eval_strategy="epoch", # Changed from evaluation_strategy to eval_strategy
    save_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=64,
    num_train_epochs=3,
    weight_decay=0.01,
    logging_dir="./logs",
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets["train"].shuffle(seed=42).select(range(20000)),
    eval_dataset=tokenized_datasets["test"].shuffle(seed=42).select(range(5000)),
    compute_metrics=compute_metrics,
    tokenizer=tokenizer
)

<ipython-input-18-eba93b47d14a>:24: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


In [15]:
import wandb # Import wandb

# Initialize wandb
wandb.init(project="imdb-sentiment-analysis") # Replace "imdb-sentiment-analysis" with your desired project name

trainer.train()

# Optional: End the wandb run when training is finished
wandb.finish()

<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize?ref=models
wandb: Paste an API key from your profile and hit enter:

 ··········


wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: nimesh2767 (nimesh2767-sardar-vallabhbhai-national-institute-of-tech) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


Epoch,Training Loss,Validation Loss,Accuracy
1,0.248700,0.197276,0.924000
2,0.123700,0.262153,0.925200
3,0.058100,0.344077,0.925400


eval/accuracy,▁▇█
eval/loss,▁▄█
eval/runtime,█▄▁
eval/samples_per_second,▁▅█
eval/steps_per_second,▁▅█
train/epoch,▁▂▃▃▄▅▅▆▇██
train/global_step,▁▂▃▃▄▅▅▆▇██
train/grad_norm,▆█▂▁▁▃▁
train/learning_rate,█▇▆▅▃▂▁
train/loss,▃█▅▃▃▁▁
eval/accuracy,0.9254


In [19]:
from transformers import DistilBertForSequenceClassification, DistilBertTokenizerFast

# Assuming 'trainer' is your Trainer instance
trainer.save_model("/content/model")  # Saves the model

# Save the tokenizer
trainer.tokenizer.save_pretrained("/content/model")


Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.


('/content/model/tokenizer_config.json',
 '/content/model/special_tokens_map.json',
 '/content/model/vocab.txt',
 '/content/model/added_tokens.json',
 '/content/model/tokenizer.json')

In [21]:
from transformers import DistilBertForSequenceClassification, DistilBertTokenizerFast
import torch

# Path to your saved model
model_path = "/content/model"

# Load model and tokenizer
model = DistilBertForSequenceClassification.from_pretrained(model_path)
tokenizer = DistilBertTokenizerFast.from_pretrained(model_path)
model.eval()

# Your input text
text = "I hate"

# Tokenize
inputs = tokenizer(text, return_tensors="pt", truncation=True, padding=True)

# Predict
with torch.no_grad():
    outputs = model(**inputs)

logits = outputs.logits
predicted_class = torch.argmax(logits, dim=-1).item()

print("Predicted class:", predicted_class)  # 0 = negative, 1 = positive


Predicted class: 0


In [22]:
!du -sh /content/model

257M	/content/model


In [24]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [25]:
import os

folder_path = "/content/drive/MyDrive/fine_tuned_models"

# Create the folder if it doesn't exist
os.makedirs(folder_path, exist_ok=True)


In [26]:
model.save_pretrained(folder_path)
tokenizer.save_pretrained(folder_path)
model.save_pretrained(folder_path)
tokenizer.save_pretrained(folder_path)


('/content/drive/MyDrive/fine_tuned_models/tokenizer_config.json',
 '/content/drive/MyDrive/fine_tuned_models/special_tokens_map.json',
 '/content/drive/MyDrive/fine_tuned_models/vocab.txt',
 '/content/drive/MyDrive/fine_tuned_models/added_tokens.json',
 '/content/drive/MyDrive/fine_tuned_models/tokenizer.json')

In [27]:
!ls /content/drive/MyDrive/fine_tuned_models

config.json	   special_tokens_map.json  tokenizer.json
model.safetensors  tokenizer_config.json    vocab.txt
